# Exercice 0 : échauffement

Dans ce TP nous allons représenter des distributions de données via différents types de graphes.

Q1: Expliquez ce que représente et quand utiliser :


*   un scatterplot
*   un barplot
*   un boxplot
*   un violinplot
*   un histogramme






```markdown
- Un scatterplot est un graphe contenant un nuage de points. Ces points peuvent être de taille variables si on leur rajoute un attribut conséquence.
- Un barplot est un graphe avec des barres selon un axe qui représente une quantité/valeur pour sa valeur situé sur l'autre axe.
- Un Boxplot est une figure avec une forme rectangulaire représentant où se situent les valeurs entre les premier et troisième quartile (on y indique aussi la médiane) et des barres sur les côtés opposés extrêmant montrant où se situent les maximum et minimum de l'ensemble des données
- Un violinplot est similaire en principe à un boxplot car il montre la médiane, moyenne, quartiles et extremas des données. Par contre, le violinplot montre aussi la répartition des données d'où sa forme souvent de violon ou de larme.
- Un histogramme est un graphe représentant l'occurence de chaque élément présent dans un ensemble de données.
```

# Exercice 1 : description de la base genbank

La base de données genbank est un des principaux dépots de données de génomes, elle gérée par le NCBI Américain (National Center for Biotechnology Information).

Dans cet exercice, allons étudier le contenu général de cette base de données en utilisant les fichiers de résumés.

---

Note technique : dans colab, vous pouvez monter votre google drive avec le code suivant :

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

---

Q1. Téléchargez les fichier "overview.txt" et "README" disponibles ici :

 https://ftp.ncbi.nlm.nih.gov/genomes/GENOME_REPORTS/



Q2. Quel est le format du fichier "overview.txt" ?

```markdown
Le fichier "overview.txt" est un .TSV soit Tab Separated Value.
```

Q3. Utilisez le fichier README pour comprendre ce que chaque colonne représente.

```markdown
1. Organism/Name: Le nom de l'espèce (par ligne)
2. Kingdom: Le groupe taxonomique de l'espèce (Archae, Bactéries, Eukaryotes, Virus)
3. Group: Groupe général de l'organisme (sous classe du royaume) comme Animal (dans Eukaryotes)
4. Sub-Group: Sous-groupe (donc encore plus spécifique) comme Insecte (pour Animal)
5. Size: Taille en Mb (Mega base) du genome de l'organisme
6. Chrs: Nombre de chromosomes de l'organisme
7. Organelles: Nombre d'organelles (un des elements composant la cellule)
8. Plasmids: Nombre de plasmides (petite molécule d'ADN circulaire pouvant se répliquer indépendemment)
9. BioProjects: Nombre de projets de séquence de génome
```

## Organisation de la base de données

Q4. Combien d’organismes sont présents ?

In [6]:
def countOrganisms():
    with open("./genome_reports/overview.txt", "r") as fp:
        return len(fp.readlines()) - 1

print(countOrganisms())

87888


### Royaumes

Q5. Qu'est-ce qu'un royaume ?

```markdown
Un règne (kingdom en anglais) est la deuxième classification du vivant en taxonomie. Il se situe sous les Domaines (ou Empire) et regroupe plusieurs embranchements dénommés phyla. Il y a actuellement 7 règnes différents:
1. Archaea
2. Bacteria
3. Protozoa
4. Chromista
5. Plantae
6. Fungi
7. Animalia
```

Q6. Combien de royaumes sont présents dans la base ?

```markdown
Dans la base seulement 4 sont présents.
```

Q7. Affichez le nombre d’organismes pour chaque royaume.

In [17]:
def countOrganismsPerKingdom():
    kingdomSizes: dict[str, int] = {}
    seenOrganisms = set()
    with open("./genome_reports/overview.txt", "r") as fp:
        lines = fp.readlines()
        for line in lines[1:]: # skip header line
            # Parse the line
            parsedLine = line.split('\t')
            # Organism is 0, Kingdom is 1
            if len(parsedLine) <= 1 or (organism := parsedLine[0]) in seenOrganisms:
                if (organism := parsedLine[0]) in seenOrganisms:
                    print("Potential error: " + organism + "has already been seen.")
                continue
            seenOrganisms.add(organism)
            kingdomSizes[parsedLine[1]] = kingdomSizes.get(parsedLine[1], 0) + 1
    return kingdomSizes
print(countOrganismsPerKingdom())

Potential error: Pseudomonas oryzihabitanshas already been seen.
{'Bacteria': 37491, 'Viruses': 32573, 'Eukaryota': 15560, 'Archaea': 2263}


### Groupes

Q8. Qu’est-ce qu’un groupe ?

```markdown
Un groupe, aussi appelé phylum, est la catégorie se situant sous un règne, comme énoncé précedemment. Il y a deux définitions pour les groupes. Il y a une définition basé sur la parenté génétique, qui requiert que les espèces appartenant à un même groupe soit au minimum étroitement apparenté, mais cette définition est peu utilisée car avec le progrès technologique, l'analyse moléculaire des génomes peut montrer que des espèces pensées apparentées ne l'était pas, nécessitant donc un changement dans la classification. De ce fait, l'autre définition se base sur un plan d'organisation, regroupant des organismes par caractères partagés par l'ensemble du groupe.
```

Q9. Combien de groupes sont présents par royaume ?

In [22]:
def countGroupsPerKingdom():
    kingdomSizes: dict[str, int] = {}
    seenGroups = set()

    with open("./genome_reports/overview.txt", "r") as fp:
        lines = fp.readlines()
        for line in lines[1:]:
            parsedLine = line.split('\t')
            if len(parsedLine) <= 2 or (group := parsedLine[2]) in seenGroups:
                continue
            seenGroups.add(group)
            kingdomSizes[parsedLine[1]] = kingdomSizes.get(parsedLine[1], 0) + 1
    return kingdomSizes

print(countGroupsPerKingdom())

{'Bacteria': 36, 'Viruses': 8, 'Eukaryota': 4, 'Archaea': 9}


Q10. Affichez pour chaque royaume le nombre d’organismes par groupe.

In [23]:
def countOrganismsByGroupPerKingdom():
    counter: dict[str, dict[str, int]] = {}
    seenOrganisms = set()

    with open("./genome_reports/overview.txt", "r") as fp:
        lines = fp.readlines()[1:]
        for line in lines:
            parsedLine = line.split('\t')
            if len(parsedLine) <= 2 or parsedLine[0] in seenOrganisms:
                continue

            kingdomName: str = parsedLine[1]
            groupName: str = parsedLine[2]
            organismName: str = parsedLine[0]

            groupCounter: dict[str, int] = counter.get(kingdomName, {})
            groupCounter[groupName] = groupCounter.get(groupName, 0) + 1
            counter[kingdomName] = groupCounter # Not efficient since duplicates the dict every time instead of just changing line
    return counter

print(countOrganismsByGroupPerKingdom())
            

{'Bacteria': {'Terrabacteria group': 13337, 'unclassified Bacteria': 346, 'FCB group': 5037, 'Pseudomonadota': 11098, 'Acidobacteriota': 256, 'Synergistota': 85, 'Bacteria incertae sedis': 3872, 'PVC group': 1091, 'Myxococcota': 146, 'Campylobacterota': 256, 'Thermodesulfobacteriota': 568, 'Spirochaetota': 344, 'Aquificota': 42, 'Thermotogota': 72, 'Atribacterota': 43, 'Other': 2, 'Bdellovibrionota': 122, 'delta/epsilon subdivisions': 288, 'Caldisericota/Cryosericota group': 21, 'Deferribacterota': 27, 'Calditrichota': 10, 'Candidatus Lernaellota': 2, 'Elusimicrobiota': 118, 'Fusobacteriota': 79, 'Candidatus Deferrimicrobiota': 2, 'Nitrospinota/Tectimicrobiota group': 39, 'Candidatus Hinthialibacterota': 2, 'Candidatus Krumholzibacteriota': 4, 'Nitrospirota': 156, 'Candidatus Moduliflexota': 2, 'Candidatus Tharpellota': 2, 'Chrysiogenota': 6, 'Coprothermobacterota': 5, 'Dictyoglomota': 5, 'environmental samples': 4, 'Thermodesulfobiota': 2, 'Thermosulfidibacterota': 1}, 'Viruses': {'Ot

## Génomes

Q11. Affichez la distribution des tailles de génomes sur toute la base de données, utilisez un boxplot ou un violinplot (avec axe en log pour y voir quelque chose).

In [61]:
import matplotlib.pyplot as plt
import pandas as pd

def chartGenomeSizeDistribution():
    filename = "./genome_reports/overview.txt"
    columnName = "Size (Mb)"
    data = pd.read_csv(filename, sep="\t", usecols=[columnName], dtype=str) # first read as string, we'll need to remove all non-floats later
    # To array
    data = data.iloc[:,0].values
    data = data[data != "-"].astype(float)

    plt.violinplot(
chartGenomeSizeDistribution()

[7.43598e-01 6.03949e-01 7.39592e-01 ... 3.20973e+01 3.96863e+01
 5.96900e-03]


Q12. Affichez la distribution par royaume.

In [ ]:
#Votre code ici

Q13. Affichez la distribution par groupe pour chaque royaume.

In [ ]:
#Votre code ici

Q14. Pour chaque royaume, donnez le nom et la taille de l’organisme avec le plus petit génome.

In [ ]:
#Votre code ici

Q15. Même chose mais pour l'organisme avec le plus grand génome.

In [ ]:
#Votre code ici

## Chromosomes

Q16. Affichez la distribution du nombre de chromosomes, sous la forme d’un histogramme, sur toute la base de données.

In [ ]:
#Votre code ici

Q17. Même chose mais pour chaque royaume sous la forme d'un boxplot (ou violinplot).

In [ ]:
#Votre code ici

Q18. Qu’observez-vous ? Comment l’expliquez-vous ?

```markdown
Votre réponse ici
```

Q19. Pour les Eucaryotes uniquement, affichez la distribution du nombre de chromosomes sous la forme d’un histogramme.

In [ ]:
#Votre code ici

Q20. Pour les Eucaryotes uniquement, affichez la taille des génomes en fonction du nombre de chromosomes, sous forme d'un graphe (scatter plot).

In [ ]:
#Votre code ici

Q21. Qu’observez-vous ?

```markdown
Votre réponse ici
```

## Plasmides

Q22. Qu'est-ce qu'un plasmide ?

```markdown
Votre réponse ici
```

Q23. Affichez le nombre moyen de plasmides par royaume, sous forme d'un barplot.

In [ ]:
#Votre code ici

Q24. Qu'observez-vous ?

```markdown
Votre réponse ici
```

## Projets

Q25. Qu'est-ce qu'un projet ?

```markdown
Votre réponse ici
```

Q26. Affichez le nombre de projets moyen par organisme pour chaque royaume, sous forme d'un barplot.

In [ ]:
#Votre code ici

Q27. Combien d'organismes ont 0 projets ?

In [ ]:
#Votre code ici

Q28. Combien d'organismes ont > 1 projets ?

In [ ]:
#Votre code ici

Q29. Affichez le nom et nombre du TOP 5 des organismes possédant le plus de projets.

In [ ]:
#Votre code ici

Q30. Expliquez rapidement pourquoi ces organismes sont les plus étudiés.

```markdown
Votre réponse ici
```

# Exercice 3 : Gènes

Pour obtenir des informations plus précises sur les gènes contenus dans les différents organismes de genbank, nous allons travailler avec le fichier de résumé d'assemblage.

Q1. Récupérez le fichier "assembly_summary_genbank_historical.txt" situé ici:
https://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/

(On utilise le fichier historique car il est plus petit, l’autre fichier fait >1GB)

Q2. Qu'est-ce que le GC content ? Pourquoi est-il mesuré?

```markdown
Votre réponse ici
```

Q3. Affichez la distribution du GC content dans tous les organismes de la liste, sous forme d'un violinplot.

In [ ]:
#Votre code ici

Q4. Qu'observez-vous ?

```markdown
Votre réponse ici
```

Q5. Affichez le GC content en fonction de la taille du génome, sous forme de scatter plot.

In [ ]:
#Votre code ici

Q6. Qu'observez-vous ?

```markdown
Votre réponse ici
```

Q7. Affichez la distribution du nombre de gènes, choisissez le plot.

In [ ]:
#Votre code ici

Q8. Qu'observez-vous ?

```markdown
Votre réponse ici
```

Q9. Enlevez les données aberrantes (outliers)

In [ ]:
#Votre code ici

Q10. Comment avez-vous fait ? Quel seuil avez-vous choisi ?

```markdown
Votre réponse ici
```

Q11. Y-a-t-il une corrélation entre taille du génome et nombre de gènes ?

In [ ]:
#Votre code ici

# Exercice 4: GC-content

Pour une séquence $s$, le contenu en GC est défini par :

$GC(s)=\frac{\#G(s) + \#C(s)}{|s|}$,

où $\#N(s)$ est le nombre de nucléotides $N$ dans la séquence $s$ et $|s|$ est la taille de $s$.

Q1. Récuperrez legénome de l’organisme *Acidianus ambivalens* à partir du FTP :

https://ftp.ncbi.nlm.nih.gov/genomes/genbank/archaea/

(dans latest_assembly_versions/GCA_009729015.1_ASM972901v1, le fichier avec l'extension "fna.gz")


Q2. Affichez le GC content global.

In [ ]:
#Votre code ici

Q3. Pour détecter des variations locales de GC, on va utiliser une fenêtre glissante de taille N nucléotides (démarrer à la position N/2 et finir à |s| -N/2).

Affichez le GC content le long du génome pour des fenêtres de tailles 50, 1000 ou 10000.

In [ ]:
#Votre code ici

Q4. Qu’observez-vous ?

```markdown
Votre réponse ici
```

Q5. On va lisser le signal pour le rendre moins bruité en utilisant la fonction *savgol_filter* de *scipy.signal* avec les paramètres $51$ et $5$.

In [ ]:
#Votre code ici

Q6. Finalement, on va rendre le code générique pour n'importe quel espèce. Modifier votre code pour qu'il prenne en entrée un nom de fichier de génome du ftp, et une taille de fenêtre, et affiche le GC content global et local.

In [ ]:
#Votre code ici